In [24]:
import os
import json

import numpy as np
import pennylane as qml

In [6]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\AHO"
file_paths = os.listdir(folder_path)

In [34]:
file_paths

['AHO_16.json', 'AHO_2.json', 'AHO_4.json', 'AHO_8.json']

In [67]:
for path in file_paths:
    data_path = os.path.join(folder_path,path)

    with open(data_path, 'r') as file:
            data = json.load(file)
    
    break

In [68]:
num_qubits = int(np.log2(data['cutoff'])) + 1
num_qubits

5

In [69]:
min_eigenvalue = min(data['exact_eigenvalues'])
min_eigenvalue

-0.0011669756810460754

In [70]:
mask = []
for d in data['min_energies']:
    if abs(d - min_eigenvalue) < 1e-6:
        mask.append(True)
    else:
        mask.append(False)


In [71]:
min_ops = [op_list for op_list, keep in zip(data['op_list'], mask) if keep]
min_ops = [eval(ops, {"RY": qml.RY, "CRX": qml.CRX, "np": np}) for ops in min_ops]

In [72]:
min_len = min([len(ops) for ops in min_ops])
min_len

6

In [73]:
reduced = [ops for ops in min_ops if len(ops) == min_len]
reduced

[[RY(np.float64(1.5707963267948966), wires=[1]),
  CRX(1.5707963267948966, wires=[0, 1]),
  RY(np.float64(1.5707963267948966), wires=[1]),
  CRX(1.5707963267948966, wires=[0, 2]),
  RY(np.float64(1.5707963267948966), wires=[2]),
  RY(np.float64(1.5707963267948966), wires=[3])],
 [RY(np.float64(1.5707963267948966), wires=[1]),
  CRX(1.5707963267948966, wires=[0, 1]),
  RY(np.float64(1.5707963267948966), wires=[1]),
  CRX(1.5707963267948966, wires=[0, 2]),
  RY(np.float64(1.5707963267948966), wires=[2]),
  RY(np.float64(1.5707963267948966), wires=[3])],
 [RY(np.float64(1.5707963267948966), wires=[1]),
  CRX(1.5707963267948966, wires=[0, 1]),
  RY(np.float64(1.5707963267948966), wires=[1]),
  CRX(1.5707963267948966, wires=[0, 2]),
  RY(np.float64(1.5707963267948966), wires=[2]),
  RY(np.float64(1.5707963267948966), wires=[3])],
 [RY(np.float64(1.5707963267948966), wires=[1]),
  CRX(1.5707963267948966, wires=[0, 1]),
  RY(np.float64(1.5707963267948966), wires=[1]),
  CRX(1.5707963267948966

In [74]:
unique = []
for op_list in reduced:
    if op_list not in unique:
        unique.append(op_list)

In [75]:
unique

[[RY(np.float64(1.5707963267948966), wires=[1]),
  CRX(1.5707963267948966, wires=[0, 1]),
  RY(np.float64(1.5707963267948966), wires=[1]),
  CRX(1.5707963267948966, wires=[0, 2]),
  RY(np.float64(1.5707963267948966), wires=[2]),
  RY(np.float64(1.5707963267948966), wires=[3])]]

In [76]:
# Define the device
dev = qml.device("default.qubit", wires=num_qubits)

# Create the QNode
@qml.qnode(dev)
def circuit():
    for op in unique[0]:
        op.queue()
    return qml.state()



In [78]:
qml.draw(circuit)()

'0: ───────────╭●──────────────────╭●──────────────────┤  State\n1: ──RY(1.57)─╰RX(1.57)──RY(1.57)─│───────────────────┤  State\n2: ───────────────────────────────╰RX(1.57)──RY(1.57)─┤  State\n3: ──RY(1.57)─────────────────────────────────────────┤  State'

In [77]:
print(qml.draw(circuit)())

0: ───────────╭●──────────────────╭●──────────────────┤  State
1: ──RY(1.57)─╰RX(1.57)──RY(1.57)─│───────────────────┤  State
2: ───────────────────────────────╰RX(1.57)──RY(1.57)─┤  State
3: ──RY(1.57)─────────────────────────────────────────┤  State
